In [1]:
%load_ext autoreload
%autoreload 2

# Proyecto final Bioinformática (NLP)

Mirar el siguiente video. Sobre todo la primer parte como para tener contexto sobre descubrimiento de drogas:

https://www.youtube.com/watch?v=jBlTQjcKuaY

Resumen y puntos importantes del video:
- Queremos entender la bioactividad de una molecúla (molecule_chembl_id) sobre una encima (Acetylcholinesterase)
- La bioactividad se medirá en este caso con el IC50 (standard_value)
- A menor IC50, menos droga para generar la misma actividad, es decir, mayor actividad relativa
- La notación de la fórmula química se llama smiles (https://en.wikipedia.org/wiki/Simplified_molecular-input_line-entry_system)
- Existen distintas técnicas para obtener features de las moléculas y en el video se describen 2:
    - Descriptores de Lipinski
    - Fingerprints del tipo pubchem
- Se construye un modelo de regresión con RandomForest para estimar el pI50 (IC50 en escala logarítimica) dado los fingerprints de entrada

### Objetivos del proyecto:
- Evaluar distintas alternativas de modelos de deep learning para resolver este problema
    - LSTM
    - CNN
    - TextCNN
- Mejorar la métrica del RandomForest
- En vez de ingresar con los features de entrada (fingerprints) como en el video, utilizar técnicas de embeddings usuales en NLP
    - Tokenización en modo caracter dado el smiles de la fórmula química
    - Utilizando un tokenizer sobre los smiles
    - Puedo usar técnicas modernas de tokenización (https://deepchem.readthedocs.io/en/2.4.0/api_reference/tokenizers.html)
- La salida a estimar por el modelo será el pIC50
- La métrica, para comparar con los resultados del RandomForest será el $R^2$

### Librerías:
- chembl-webresource-client: Para bajar el dataset (https://pypi.org/project/chembl-webresource-client/)
- deepchem: libería muy interesante con muchas implementaciones de deep learning aplicadas a la química (https://github.com/deepchem/deepchem)

In [2]:
import pandas as pd
from chembl_webresource_client.new_client import new_client

# Cliente API

En esta notebook solo se baja el dataset. No tiene que hacer nada más que ejecutarla y entenderla

Librería para baja el dataset

In [3]:
target = new_client.target
target_query = target.search('acetylcholinesterase')
targets = pd.DataFrame.from_dict(target_query)

In [4]:
targets.head()

,cross_references,organism,pref_name,score,species_group_flag,target_chembl_id,target_components,target_type,tax_id
0,"[{'xref_id': 'P22303', 'xref_name': None, 'xre...",Homo sapiens,Acetylcholinesterase,27.0,False,CHEMBL220,"[{'accession': 'P22303', 'component_descriptio...",SINGLE PROTEIN,9606
1,[],Homo sapiens,Cholinesterases; ACHE & BCHE,27.0,False,CHEMBL2095233,"[{'accession': 'P06276', 'component_descriptio...",SELECTIVITY GROUP,9606
2,[],Drosophila melanogaster,Acetylcholinesterase,18.0,False,CHEMBL2242744,"[{'accession': 'P07140', 'component_descriptio...",SINGLE PROTEIN,7227
3,[],Bemisia tabaci,AChE2,16.0,False,CHEMBL2366409,"[{'accession': 'B3SST5', 'component_descriptio...",SINGLE PROTEIN,7038
4,[],Leptinotarsa decemlineata,Acetylcholinesterase,16.0,False,CHEMBL2366490,"[{'accession': 'Q27677', 'component_descriptio...",SINGLE PROTEIN,7539


In [5]:
selected_target = targets.target_chembl_id[0]
selected_target
# 'CHEMBL220'

'CHEMBL220'

In [6]:
activity = new_client.activity
res = activity.filter(target_chembl_id=selected_target).filter(standard_type="IC50")

# Bajada de data
Puede tardar un poco dependiendo de que tan saturado este el server

Por eso el for, para ver el progreso y bajar la ansiedad. Son en el orden de 7500K

In [7]:
res_cols = []
for i, r in enumerate(res):
    print(f'{i}\r', end='')
    res_cols.append(r)

In [8]:
df = pd.DataFrame(res_cols)

In [9]:
df

,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,...,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,33969,[],CHEMBL643384,Inhibitory concentration against acetylcholine...,B,None,None,BAO_0000190,BAO_0000357,...,Homo sapiens,Acetylcholinesterase,9606,None,None,IC50,uM,UO_0000065,None,0.75
1,None,37563,[],CHEMBL643384,Inhibitory concentration against acetylcholine...,B,None,None,BAO_0000190,BAO_0000357,...,Homo sapiens,Acetylcholinesterase,9606,None,None,IC50,uM,UO_0000065,None,0.1
2,None,37565,[],CHEMBL643384,Inhibitory concentration against acetylcholine...,B,None,None,BAO_0000190,BAO_0000357,...,Homo sapiens,Acetylcholinesterase,9606,None,None,IC50,uM,UO_0000065,None,50.0
3,None,38902,[],CHEMBL643384,Inhibitory concentration against acetylcholine...,B,None,None,BAO_0000190,BAO_0000357,...,Homo sapiens,Acetylcholinesterase,9606,None,None,IC50,uM,UO_0000065,None,0.3
4,None,41170,[],CHEMBL643384,Inhibitory concentration against acetylcholine...,B,None,None,BAO_0000190,BAO_0000357,...,Homo sapiens,Acetylcholinesterase,9606,None,None,IC50,uM,UO_0000065,None,0.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8390,None,23311841,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL4844259,Inhibition of human recombinant AChE using ace...,B,None,None,BAO_0000190,BAO_0000357,...,Homo sapiens,Acetylcholinesterase,9606,None,None,IC50,nM,UO_0000065,None,1.07
8391,Not Determined,23311842,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL4844259,Inhibition of human recombinant AChE using ace...,B,None,None,BAO_0000190,BAO_0000357,...,Homo sapiens,Acetylcholinesterase,9606,None,None,IC50,None,None,None,None
8392,None,23311843,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL4844259,Inhibition of human recombinant AChE using ace...,B,None,None,BAO_0000190,BAO_0000357,...,Homo sapiens,Acetylcholinesterase,9606,None,None,IC50,nM,UO_0000065,None,26.8
8393,Not Determined,23311844,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL4844259,Inhibition of human recombinant AChE using ace...,B,None,None,BAO_0000190,BAO_0000357,...,Homo sapiens,Acetylcholinesterase,9606,None,None,IC50,None,None,None,None


# Limpio data

In [10]:
df = df.dropna(subset=['standard_value', 'canonical_smiles'])
df = df.drop_duplicates(['canonical_smiles'])

In [11]:
selection = ['molecule_chembl_id','canonical_smiles','standard_value']
df = df[selection]

In [12]:
df

,molecule_chembl_id,canonical_smiles,standard_value
0,CHEMBL133897,CCOc1nn(-c2cccc(OCc3ccccc3)c2)c(=O)o1,750.0
1,CHEMBL336398,O=C(N1CCCCC1)n1nc(-c2ccc(Cl)cc2)nc1SCC1CC1,100.0
2,CHEMBL131588,CN(C(=O)n1nc(-c2ccc(Cl)cc2)nc1SCC(F)(F)F)c1ccccc1,50000.0
3,CHEMBL130628,O=C(N1CCCCC1)n1nc(-c2ccc(Cl)cc2)nc1SCC(F)(F)F,300.0
4,CHEMBL130478,CSc1nc(-c2ccc(OC(F)(F)F)cc2)nn1C(=O)N(C)C,800.0
...,...,...,...
8384,CHEMBL4859103,CC1=CC2Cc3nc4cc(Cl)ccc4c(NCCCCCCCCCNC(=O)c4cc(...,2.57
8385,CHEMBL4863615,CC1=CC2Cc3nc4cc(Cl)ccc4c(NCCCCCCCCC(=O)Nc4cn[n...,2.39
8386,CHEMBL4854913,CC1=CC2Cc3nc4cc(Cl)ccc4c(NCCCCCCCCC(=O)N4CCNC(...,0.41
8387,CHEMBL4848527,CC1=CC2Cc3nc4cc(Cl)ccc4c(NCCCCCCCCC(=O)NCc4cc[...,0.63


# Preprocesamiento y normalización

In [13]:
import numpy as np

In [14]:
df['standard_value'] = df['standard_value'].apply(pd.to_numeric)

In [15]:
# En el video se hace esta normalización. No la veo del todo necesaria
df['standard_value_norm'] = df['standard_value'].apply(lambda x: (x>1e8)*1e8 + (x<=1e8)*x)

In [16]:
df['pIC50'] = df['standard_value'].apply(lambda x: -np.log10(x*(10**-9)))

In [17]:
# Elimino datos no numericos de la columna target. De otro modo luego ocasionan inconvenientes durante 
# el proceso de entrenamiento
df = df[~df.isin([np.nan, np.inf, -np.inf]).any(1)]

/var/folders/cr/lxlcgg7n0c36k6ls750lqw_w0000gp/T/ipykernel_22867/2445790529.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  df = df[~df.isin([np.nan, np.inf, -np.inf]).any(1)]


In [18]:
df

,molecule_chembl_id,canonical_smiles,standard_value,standard_value_norm,pIC50
0,CHEMBL133897,CCOc1nn(-c2cccc(OCc3ccccc3)c2)c(=O)o1,750.00,750.00,6.124939
1,CHEMBL336398,O=C(N1CCCCC1)n1nc(-c2ccc(Cl)cc2)nc1SCC1CC1,100.00,100.00,7.000000
2,CHEMBL131588,CN(C(=O)n1nc(-c2ccc(Cl)cc2)nc1SCC(F)(F)F)c1ccccc1,50000.00,50000.00,4.301030
3,CHEMBL130628,O=C(N1CCCCC1)n1nc(-c2ccc(Cl)cc2)nc1SCC(F)(F)F,300.00,300.00,6.522879
4,CHEMBL130478,CSc1nc(-c2ccc(OC(F)(F)F)cc2)nn1C(=O)N(C)C,800.00,800.00,6.096910
...,...,...,...,...,...
8384,CHEMBL4859103,CC1=CC2Cc3nc4cc(Cl)ccc4c(NCCCCCCCCCNC(=O)c4cc(...,2.57,2.57,8.590067
8385,CHEMBL4863615,CC1=CC2Cc3nc4cc(Cl)ccc4c(NCCCCCCCCC(=O)Nc4cn[n...,2.39,2.39,8.621602
8386,CHEMBL4854913,CC1=CC2Cc3nc4cc(Cl)ccc4c(NCCCCCCCCC(=O)N4CCNC(...,0.41,0.41,9.387216
8387,CHEMBL4848527,CC1=CC2Cc3nc4cc(Cl)ccc4c(NCCCCCCCCC(=O)NCc4cc[...,0.63,0.63,9.200659


In [19]:
! mkdir data

mkdir: data: File exists


In [20]:
df.to_csv('data/acetylcholinesterase_02_bioactivity_data_preprocessed.csv', index=False)